In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import json
import logging
import os
import re
import sys
from functools import partial
from itertools import chain
from pathlib import Path
from threading import Thread
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import QuantLib as ql
from bhutils import bhutilspy as bh
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.ezutils import (
    SQL,
    COBCurvesRemote,
    COBInflLocal,
    LiveCurvesRemote,
    LiveInflLocal,
    ZCILSHelper,
    bh,
    bhs,
    get_id,
    get_price_info,
)
from gioutils.gui.base import Tabs, View, ViewStore, WidgetView
from gioutils.utils import (
    RepeatedTimer,
    RepeatedTimerThread,
    get_bday,
    get_next_n,
    parse_offset,
    today,
)
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from api.app import DataCollector
from api.collectors import (
    ECO,
    CollectorStoreMixin,
    ECOPred,
    Futures,
    FwdMatrix,
    Market,
    Mtgs,
    SpotForward,
    VolMatrix,
    ZCILMatrix,
)
from api.data.base import data_path, radar_path
from api.gui.editable_grid import EditableGrid
from api.gui.global_config import GlobalConfig
from api.gui.grids import (  # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
    FutureGrid,
    GroupMktGrid,
    MatGrid,
    MtgGrid,
    SpotGrid,
)
from api.gui.monitors import ECOMonitor, MktMonitor, VolMonitor
from api.gui.pandas import DataGridGio, DFOutput, custom_styler_row_line, negative_red
from api.gui.params import bbg_params, fwd_params, spot_params
from api.gui.price_grids import PricingGrid
from api.gui.renderers import RendererComponentStoreMixin, RendererStoreMixin
from api.gui.theme import bg_color, grid_style

# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens


pd.options.plotting.backend = "plotly"

In [5]:
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()

In [6]:
ccys = [
    "EUR",
    "USD",
    "GBP",
    # 'JPY'
]

In [7]:
live = LiveCurvesRemote(ccys=ccys)

In [8]:
offsets = ["1d", "1w", "2w", "1m", "3m", "6m", "ytd", "1y"]
generators = ["USD SOFR", "EUR ESTER", "GBP SONIA"]

In [ ]:
def get_periods(n):
    if not isinstance(n, (list, np.ndarray, pd.Series)):
        n = [n]
    p = []
    for n_ in n:

        dec = n_ % 1
        if not n_:
            p.append(0)
        elif dec:
            p.append(f"{round(n_ * 12)}m")
        else:
            p.append(f"{round(n_)}y")
    return p

In [ ]:
starts = [
    0,
    1 / 12,
    2 / 12,
    0.25,
    0.5,
    0.75,
    1,
    1.5,
    *range(2, 11),
    12,
    15,
    20,
    25,
    30,
    35,
    40,
    50,
]

starts = get_periods(starts)

In [ ]:
tenors = [
    1 / 12,
    2 / 12,
    0.25,
    0.5,
    0.75,
    1,
    1.5,
    *range(2, 11),
    12,
    15,
    20,
    25,
    30,
    35,
    40,
    50,
]

tenors = get_periods(tenors)

In [ ]:
curve_periods = ["live", *[offset for offset in offsets]]

In [ ]:
curves = {
    "live": live,
    **{offset: COBCurvesRemote(ccys=ccys, offset=offset) for offset in offsets},
}

In [ ]:
from itertools import combinations, product

In [ ]:
swaps = [*product(starts, tenors)]

In [ ]:
spreads = [*combinations(swaps, 2)]
flies = [*combinations(swaps, 3)]

In [ ]:
spread_wgt = [-1, 1]
fly_wgt = [-1, 2, -1]

In [ ]:
fwd_spreads = pd.DataFrame([*combinations(tenors, 2)], columns=['leg1', 'leg2'])
fwd_flies = pd.DataFrame([*combinations(tenors, 3)], columns=['leg1', 'leg2', 'leg3'])

In [ ]:
rates = {
    key: bh.bhFiRates(
        fwdterms=starts,
        tenors=tenors,
        generatorname="USD SOFR",
        curves=curve.get_snap(),
        collateral="LCH",
        generators=bhs.gens,
    ).py
    for key, curve in curves.items()
}
rates = pd.concat(rates)

In [ ]:
rates = rates.rename({'in.eff': 'start', 'in.mty': 'tenor'}, axis=1)
rates['start'] = pd.Categorical(rates['start'], categories=starts, ordered=True)
rates['tenor'] = pd.Categorical(rates['tenor'], categories=tenors, ordered=True)

In [ ]:
rates_ = rates.set_index(['start', 'tenor'], append=True).droplevel(1)

In [ ]:
flat_spreads = {}
for period in curve_periods:
    for start in starts:
        for index, legs in fwd_spreads.iterrows():
            flat_spreads[tuple([period, start, *legs])] = rates_.loc[(period, start, [*legs]), 'rate'] @ spread_wgt

In [ ]:
flat_spreads_ = (pd.Series(flat_spreads, name='value') * 10000).rename_axis(['period', 'start', 'leg1', 'leg2']).reset_index() 

In [ ]:
flat_spreads_['period'] = pd.Categorical(flat_spreads_['period'], categories=curve_periods, ordered=True)
flat_spreads_['start'] = pd.Categorical(flat_spreads_['start'], categories=starts, ordered=True)
flat_spreads_['leg1'] = pd.Categorical(flat_spreads_['leg1'], categories=tenors, ordered=True)
flat_spreads_['leg2'] = pd.Categorical(flat_spreads_['leg2'], categories=tenors, ordered=True)

In [ ]:
flat_spreads_.to_clipboard()

In [ ]:
flat_spreads_.columns

In [ ]:
key = '1d'
(flat_spreads_.set_index(['period', 'start', 'leg1', 'leg2']).loc['live'] - flat_spreads_.set_index(['period', 'start', 'leg1', 'leg2']).loc[key]).to_clipboard()

In [ ]:
flat_spreads_movers = flat_spreads_.set_index(['period', 'start', 'leg1', 'leg2']).loc['live'].squeeze().rename('live')
flat_spreads_movers = pd.concat([flat_spreads_movers, *[(flat_spreads_movers - flat_spreads_.set_index(['period', 'start', 'leg1', 'leg2']).loc[key].squeeze()).rename(key) for key in curve_periods[1:]]], axis=1).dropna()

In [ ]:
top = 20
top_steepener = pd.concat({key: flat_spreads_movers.sort_values(by=key, ascending=False)[key].head(top) for key in curve_periods}).rename('value')
top_flatteners = pd.concat({key: flat_spreads_movers.sort_values(by=key, ascending=False)[key].head(top) for key in curve_periods}).rename('value')

In [ ]:
top_steepener.loc[curve_periods[0]].to_clipboard()

In [ ]:
(flat_spreads_.set_index(['period', 'start', 'leg1', 'leg2']).loc['live'] - flat_spreads_.set_index(['period', 'start', 'leg1', 'leg2']).loc[key]).sort_values(by='value', ascending=False).dropna().head(20)

In [ ]:
flat_spreads_.query(f'period=="live"').pivot(columns='start', index=['leg1', 'leg2'], values='value').to_clipboard()

In [ ]:
flat_spreads_.query(f'period=="live"').pivot(columns='start', index=['leg1', 'leg2'], values='value')

In [ ]:
flat_spreads_pivot = flat_spreads_.pivot(columns='start', index=['period', 'leg1', 'leg2'], values='value')

In [ ]:
(flat_spreads_pivot.loc['live'] - flat_spreads_pivot.loc['3m']).to_clipboard()

In [ ]:
pd.MultiIndex.from_frame(rates[["in.eff", "in.mty"]].drop_duplicates())

In [ ]:
rates_pivot = rates.rename_axis(["period", None]).reset_index(level=0).pivot(
    index=["in.eff", "in.mty"], columns="period", values="rate"
).loc[pd.MultiIndex.from_frame(rates[["in.eff", "in.mty"]].drop_duplicates()), [*curves]]

In [ ]:
rates_changes = - rates_pivot.drop('live', axis=1).subtract(rates_pivot['live'], axis=0).dropna()

In [ ]:
rates_changes.index

In [ ]:
rates_changes.rank(pct=True).mean(axis=1).sort_values(ascending=False)[:50]

In [ ]:
(600 * 4 -1)  ** 2

In [ ]:
rates_pivot.diff(axis=1)

In [ ]:
rates_pivot.drop('live', axis=1) rates_pivot['live'].to_numpy()

In [ ]:
rates_pivot['live'].to_numpy().shape

In [ ]:
rates_pivot['live'].to_frame()

In [ ]:
rates_pivot.drop('live', axis=1) - rates_pivot['live']

In [ ]:
x_sum = x.sum(axis=-1)
x_bool = (x_sum[:, 0] < x_sum[:, 1]) & (x_sum[:, 1] < x_sum[:, 2])

In [ ]:
x_filtered = x[x_bool.nonzero()[0], ...]

In [ ]:
x_filtered

In [ ]:
_.shape

In [ ]:
len(flies)

In [ ]:
dc = DataCollector(
    live_infl=live_infl,
    cob_infl=cob_infl,
    ccys=None,
    bq=bq,
)

In [ ]:
self = dc

In [ ]:
threads = []
res = {}
for i in range(2):
    thread = Thread(target=dl, args=(bq, i, res), daemon=True)
    thread.start()
    threads.append(thread)

In [ ]:
snaps = self.get_snaps()
bq = self.bq.start()


threads = []
print("start for")
for name, collector in self.collectors.items():
    # kwargs = collector.get_collector_kwargs()
    thread = Thread(
        target=collector.get_data,
        kwargs=dict(snaps=snaps, cob_date=self.cob_curves.date, bq=bq),
        daemon=True,
    )
    thread.start()
    threads.append(thread)
print("end for")
[thread.join() for thread in threads]
print("end join")

In [ ]:
gc = GlobalConfig(dc=dc, timer=RepeatedTimerThread)

In [ ]:
tabs = Tabs(
    {
        # 'radar': v.Container(
        #     fluid=True,
        #     children=[
        #         v.Row(children=[v.Col(children=[gc.view])]),
        #         v.Row(children=[v.Col(children=[mm.view])])
        #     ],
        # ),
        "radar": v.Container(fluid=True, children=[mm.view]),
        "vol": v.Container(fluid=True, children=[vm.view]),
        "eco": v.Container(fluid=True, children=[em.view]),
        "pricer": v.Container(
            fluid=True,
            children=[w.VBox([pricer_grid.box, pricer_grid.price_grid.view])],
        ),
        "monitor": pricer_grid.expr_dg.view,
        # 'a': w.HTML('test'),
        # 'b': w.HTML('test'),
        # 'c': w.HTML('test'),
    },
    force_grid_render=False,
    # tabs_kwargs={'class_': "mb-2"}
)

In [ ]:
v.Container(
    children=[
        # gc.view,
        v.Row(
            children=[
                v.Col(children=[gc.view], offset=6, class_="my-0 py-0"),
            ],
        ),
        v.Row(
            children=[
                v.Col(children=[tabs.view], class_="ma-0 pa-0"),
            ],
        ),
    ],
    fluid=True,
    class_="ma-0 pa-0",
)